In [4]:
from tensorflow.keras.models import load_model

import pandas as pd
import numpy as np
import plotly.express as px

In [5]:
MODEL_DIR = './models/'
STATS_DIR = './stats/'

In [6]:
name = input('Enter model name: ')

kmodel = load_model(MODEL_DIR + name + '_keras')
print('Loaded model: ' + name)

Loaded model: 2k


In [7]:
from sklearn.linear_model import LinearRegression
X = pd.read_csv(STATS_DIR + 'X.csv')
y = pd.read_csv(STATS_DIR + 'y.csv')
X.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_PTS_PER_100'], inplace=True)
X = (X - X.mean()) / X.std()

X = X.to_numpy()
y = y['PLAYER_PTS'].to_numpy()

lmodel = LinearRegression().fit(X=X, y=y)

In [8]:
# Predict matchup
def predictMatchup(off_id, def_id, model):
    # Get offense stats
    data = pd.read_csv('./stats/2021-22_off_stats.csv')
    data = data.add_prefix('OFF_')
    data.rename(columns={'OFF_SEASON_ID': 'SEASON_ID'}, inplace=True)
    inputs_o = data[data['OFF_PLAYER_ID'] == off_id]

    # Get defensive stats
    def_data = pd.read_csv('./stats/2021-22_def_stats.csv')
    def_data = def_data.fillna(0)
    def_data = def_data.add_prefix('DEF_')
    def_data.rename(columns={'DEF_SEASON_ID': 'SEASON_ID', 'DEF_DEF_RATING': 'DEF_RATING', 
                        'DEF_DEF_BOXOUTS': 'DEF_BOXOUTS'}, inplace=True)
    inputs_d = def_data[def_data['DEF_PLAYER_ID'] == def_id]
    # print('Offensive player: ' + def_data[def_data['DEF_PLAYER_ID'] == off_id].reset_index()['DEF_PLAYER_NAME'][0])
    # print('Devensive player: ' + def_data[def_data['DEF_PLAYER_ID'] == def_id].reset_index()['DEF_PLAYER_NAME'][0])

    # Get both inputs together
    inputs_o.drop(columns=['OFF_PLAYER_ID', 'OFF_PTS_PER_100', 'OFF_PTS', 'OFF_TOUCHES'], inplace=True)
    inputs_d.drop(columns=['DEF_PLAYER_ID', 'DEF_PLAYER_NAME', 'SEASON_ID'], inplace=True)
    inputs_o.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_FGM', 'OFF_EFF_FG_PCT', 'OFF_OREB', 'OFF_PAINT_TOUCH_FGA', 'OFF_PAINT_TOUCH_FGM', 'OFF_PAINT_TOUCH_TOV', 'OFF_PAINT_TOUCH_PASSES', 'OFF_PAINT_TOUCH_PTS', 'OFF_DREB', 'OFF_FG3M', 'OFF_FTM'], inplace=True)
    inputs_d.drop(columns=['DEF_D_FGM', 'DEF_OPP_PTS_PAINT', 'DEF_CONTESTED_SHOTS_2PT', 'DEF_DEFLECTIONS', 'DEF_OPP_PTS_FB', 'DEF_OPP_PTS_2ND_CHANCE', 'DEF_OPP_PTS_OFF_TOV', 'DEF_PCT_PLUSMINUS', 'DEF_G'], inplace=True)

    inputs_o = inputs_o.reset_index(drop=True)
    inputs_d = inputs_d.reset_index(drop=True)
    X_predict = pd.concat([inputs_d, inputs_o], axis=1)

    X_predict.sort_index(axis=1, inplace=True)

    # Normalize inputs
    X_seqtest = pd.read_csv('./stats/X.csv')
    X_seqtest.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_PTS_PER_100'], inplace=True)
    X_seqtest.drop(columns=['OFF_MIN', 'OFF_GP', 'DEF_MIN', 'DEF_GP'], inplace=True)
    # X_seqtest = X_seqtest[X_seqtest['SEASON_ID'] == 22021]

    X_predict = (X_predict - X_seqtest.mean()) / X_seqtest.std()

    X_predict = X_predict.fillna(0)

    X_predict = X_predict.sort_index(axis=1)

    # X_predict.columns
    y = model.predict(X_predict.to_numpy())
    return y

In [9]:
team_df = pd.read_csv('./stats/2021-22_playoffs_team_info.csv')
h2h_df = pd.read_csv('./stats/2021-22_playoffs_h2h_stats.csv')
off_data = pd.read_csv('./stats/2021-22_playoffs_off_stats.csv')

In [10]:
def predictPointsTeam(off_id, def_team, model):
    off_times = h2h_df[h2h_df['OFF_PLAYER_ID'] == off_id]
    def_players = team_df[team_df['TEAM_ID'] == def_team]['PLAYER_ID'].values.tolist()

    # Get matchup mins against team
    truths = []
    for id in off_times['DEF_PLAYER_ID']:
        if id in def_players:
            truths.append(True)
        else:
            truths.append(False)
    off_times = off_times[truths]
    off_times = off_times[off_times['MATCHUP_TIME_SEC'] > 120]

    # Predict possessions
    poss_data = off_data[off_data['PLAYER_ID'] == off_id].reset_index()
    poss = (poss_data['PTS'] / (poss_data['PTS_PER_100'] / 100))[0]

    predictions = [predictMatchup(off_id, def_id, model) for def_id in off_times['DEF_PLAYER_ID']]
    off_times['PREDICTED_POINTS'] = predictions

    # Calculate overall prediction
    tot_min = off_times['MATCHUP_TIME_SEC'].sum()
    tot = 0
    for row in off_times[['MATCHUP_TIME_SEC', 'PREDICTED_POINTS']].to_numpy():
        tot += row[0] / tot_min * row[1]
    
    return tot

## Make predictions

Take players from array and predict their points scored against the given team

In [ ]:
matchups = [[201939, 1610612763], # Curry vs Grizzlies
            [202691, 1610612763], # Klay vs Grizzlies
            [1628991, 1610612744], # Jaren Jackson Jr vs Warriors
            [203937, 1610612744], # Kyle Anderson vs Warriors
            ]

for row in matchups:
    row.append(predictPointsTeam(row[0], row[1], kmodel)[0][0])
    row.append(predictPointsTeam(row[0], row[1], lmodel)[0])

c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\Darrick\anaco

Add real stats

In [12]:
matchups[0].append(14)
matchups[1].append(19)
matchups[2].append(21)
matchups[3].append(9)

In [13]:
df = pd.DataFrame(matchups, columns=['PLAYER_ID', 'TEAM_ID', 'KERAS_PTS', 'LR_PTS', 'ACTUAL_PTS'])
df

,PLAYER_ID,TEAM_ID,KERAS_PTS,LR_PTS,ACTUAL_PTS
0,201939,1610612763,22.934444,24.203242,14
1,202691,1610612763,22.845484,25.091528,19
2,1628991,1610612744,23.121487,21.806642,21
3,203937,1610612744,20.381720,18.846893,9


In [15]:
import plotly.graph_objects as go

players = ['S. Curry', 'K. Thompson', 'J. Jackson Jr.', 'K. Anderson']

fig = go.Figure(data=[
    go.Bar(name='Actual Points', x=players, y=df['ACTUAL_PTS']),
    go.Bar(name='Model\'s Predicted Points', x=players, y=df['KERAS_PTS']),
    go.Bar(name='LR Predicted Points', x=players, y=df['LR_PTS'])
])
fig.update_layout(barmode='group')
fig.show()

In [30]:
matchups = [[202710, 1610612755], # Jimmy vs 76ers
            [1629639, 1610612755], # Herro vs 76ers
            [203954, 1610612748], # Embiid vs Miami
            [202699, 1610612748], # Tobias Harris vs Miami
            ]

for row in matchups:
    row.append(predictPointsTeam(row[0], row[1], kmodel)[0][0])
    row.append(predictPointsTeam(row[0], row[1], lmodel)[0])

c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCo

In [31]:
matchups[0].append(32)
matchups[1].append(10)
matchups[2].append(20)
matchups[3].append(14)

In [32]:
df = pd.DataFrame(matchups, columns=['PLAYER_ID', 'TEAM_ID', 'KERAS_PTS', 'LR_PTS', 'ACTUAL_PTS'])
df

players = ['J. Butler', 'T. Herro', 'J. Embiid', 'T. Harris']

fig = go.Figure(data=[
    go.Bar(name='Actual Points', x=players, y=df['ACTUAL_PTS']),
    go.Bar(name='Model\'s Predicted Points', x=players, y=df['KERAS_PTS']),
    go.Bar(name='LR Predicted Points', x=players, y=df['LR_PTS'])
])
fig.update_layout(barmode='group')
fig.show()

In [36]:
matchups = [[1626164, 1610612742],
            [101108, 1610612742],
            [1629029, 1610612756],
            [1628973, 1610612756],
            ]

for row in matchups:
    row.append(predictPointsTeam(row[0], row[1], kmodel)[0][0])
    row.append(predictPointsTeam(row[0], row[1], lmodel)[0])

c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCo

In [37]:
matchups[0].append(19)
matchups[1].append(13)
matchups[2].append(33)
matchups[3].append(18)

In [39]:
df = pd.DataFrame(matchups, columns=['PLAYER_ID', 'TEAM_ID', 'KERAS_PTS', 'LR_PTS', 'ACTUAL_PTS'])
df

players = ['D. Booker', 'C. Paul', 'L. Doncic', 'J. Brunson']

fig = go.Figure(data=[
    go.Bar(name='Actual Points', x=players, y=df['ACTUAL_PTS']),
    go.Bar(name='Model\'s Predicted Points', x=players, y=df['KERAS_PTS']),
    go.Bar(name='LR Predicted Points', x=players, y=df['LR_PTS'])
])
fig.update_layout(barmode='group')
fig.show()